In [12]:
import numpy as np
import pandas as pd
import time

data = pd.read_csv("datasets/train.csv")
x_train, y_train = data.iloc[:int(len(data) * 0.9), 1:], data.iloc[:int(len(data) * 0.9), 0]
x_valid, y_valid = data.iloc[int(len(data) * 0.9):, 1:], data.iloc[int(len(data) * 0.9):, 0]

In [16]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

classifiers = {"random": RandomForestClassifier(n_estimators = 150, random_state = 42),
               "extra": ExtraTreesClassifier(n_estimators = 150, random_state = 42),
               "svc": SVC(kernel = "rbf", random_state = 42)}

for name, model in classifiers.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    print(name, accuracy_score(model.predict(x_valid), y_valid), "%.2f seconds" %(time.time() - start_time))

random 0.9657142857142857 29.44 seconds
extra 0.9666666666666667 22.58 seconds
svc 0.974047619047619 83.43 seconds


In [23]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators = [("random", RandomForestClassifier(n_estimators = 150, random_state = 42)),
                                           ("extra", ExtraTreesClassifier(n_estimators = 150, random_state = 42)),
                                           ("svc", SVC(kernel = "rbf", random_state = 42))])
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('random',
                              RandomForestClassifier(n_estimators=150,
                                                     random_state=42)),
                             ('extra',
                              ExtraTreesClassifier(n_estimators=150,
                                                   random_state=42)),
                             ('svc', SVC(random_state=42))])

In [27]:
voting_clf.score(x_valid, y_valid)

0.9697619047619047

In [30]:
voting_clf.voting = "soft"
voting_clf.named_estimators["svc"].probability = True
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('random',
                              RandomForestClassifier(n_estimators=150,
                                                     random_state=42)),
                             ('extra',
                              ExtraTreesClassifier(n_estimators=150,
                                                   random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [31]:
voting_clf.score(x_valid, y_valid)

0.9757142857142858

In [33]:
x_test = pd.read_csv("datasets/test.csv")
predictions = voting_clf.predict(x_test)

ValueError: All arrays must be of the same length

In [41]:
submission = pd.DataFrame({"ImageId" : range(1, 28001), "Label" : predictions})

In [42]:
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [43]:
submission.to_csv("submissionEnsemble2.csv", index = False)